In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Functional LUME-Astra examples

This is the functional way to run astra and return the evaluate Astra object, or simple dict of outputs

Settings is a list of settings that can appear in the input file.

In [2]:
from astra import run_astra, run_astra_with_generator, template_dir

import matplotlib.pyplot as plt
import os

In [3]:
# Input template file 
ASTRA_IN = os.path.join(template_dir, 'dcgun/astra.in')

In [ ]:
settings0 = {'zstop':1, 'zemit':200, 'zphase':1, 'phases':True}

# Just run astra
A1 = run_astra(settings0, astra_input_file=ASTRA_IN, verbose=True, timeout=100)

run_astra
zstop is in astra newrun
zemit is in astra newrun
zphase is in astra newrun
phases is in astra newrun
Linked /Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun/generator.part to /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmph7le2r0q/generator.part
Linked /Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun/dcgun_GHV.dat to /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmph7le2r0q/dcgun_GHV.dat
Linked /Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun/solenoid_SLA_L60.dat to /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmph7le2r0q/solenoid_SLA_L60.dat


In [ ]:
GENERATOR_IN = os.path.join(template_dir, 'dcgun/generator.in')

# Run Astra with Generator
settings0['ipart']= 2000

A2 = run_astra_with_generator(settings0, astra_input_file=ASTRA_IN,
                              generator_input_file=GENERATOR_IN, verbose=True)

In [ ]:
from astra.astra_distgen import run_astra_with_distgen
DISTGEN_IN = os.path.join(template_dir, 'dcgun/distgen.yaml')

# Run Astra with Distgen
settings0 = {'zstop':1, 'zemit':200, 'zphase':1, 'phases':True}
settings0['distgen:n_particle'] = 2000

A3 = run_astra_with_distgen(settings0, astra_input_file=ASTRA_IN,
                              distgen_input_file=DISTGEN_IN, verbose=True)

In [ ]:
# These Generator and Distgen inputs are set up to produce nearly the same initial beams

plt.plot(A2.stat('sigma_x'))
plt.plot(A3.stat('sigma_x'))

# Evaluate functions

For scans an optimizations, a user often wants to run a simulation many times and examine some particular output. The package provides several `evaluate_` functions that are similar to the `run_` functions above, but apply some merit function to the output and returns that. Additionally, if an `archive_path` is provided, the `.archive` method will be called to save the complete object output

In [ ]:
# This is the default merit function that is applied. The user can supply a different function if needed.

from astra.evaluate import default_astra_merit

default_astra_merit(A2)

# evaluate_astra_with_generator

In [ ]:
# An even simpler run, returns a simple dict of outputs. For use in optimization
from astra import evaluate_astra_with_generator

MY_GENERATOR_TEMPLATE = os.path.join(template_dir, 'dcgun/generator.in')

# Run Astra with Generator
# Run Astra with Generator
settings0 = {'zstop':1, 'zemit':200, 'zphase':1, 'phases':True}
settings0['ipart'] = 2000

outputs1 = evaluate_astra_with_generator(settings0, 
                                        astra_input_file=ASTRA_IN,
                                        generator_input_file=GENERATOR_IN,
                                        archive_path = '.')
outputs1

In [ ]:
# The archive can be loaded into a clean object.
from astra import Astra
AX = Astra()
AX.load_archive(outputs1['archive'])

# Check the merit
default_astra_merit(AX)

In [ ]:
# Cleanup
os.remove(outputs1['archive'])

# evaluate_astra_with_distgen

In [ ]:
from astra import evaluate_astra_with_distgen

?evaluate_astra_with_distgen

In [ ]:
# Run Astra with Distgen
settings5 = {'zstop':1, 'zemit':200, 'zphase':1, 'phases':True}
settings5['distgen:n_particle'] = 2000

outputs2 = evaluate_astra_with_distgen(settings5, astra_input_file=ASTRA_IN,
                              distgen_input_file=DISTGEN_IN, archive_path='.')
outputs2

In [ ]:
from distgen import Generator

In [ ]:
# Re-form the generator object
G = Generator()
G.load_archive(outputs2['archive'])
G

In [ ]:
# Note that there are no particles, this is just the input. 
# Particles are 
G.particles == None

In [ ]:
# This will essentially recreate evaluate_astra_with_distgen
A = Astra(verbose=False)

A.load_archive(outputs2['archive'])

# Make particles
G.run()

A.initial_particles = G.particles
A.configure()
A.run()
default_astra_merit(A)

In [ ]:
# Cleanup
os.remove(outputs2['archive'])

In [ ]:
# Compare Generator, Distgen. Thee should be similar, but not exactly the same. 
for k in outputs1:
    print(k, outputs1[k], outputs2[k])